In [ ]:
from backend_algo import init_main_officers_template, generate_main_officers_schedule, officer_to_counter_matrix, find_partial_availability, build_officer_schedules, generate_break_schedules, generate_sos_schedule_matrix, greedy_smooth_schedule_beam, greedy_longest_partition_inclusive, get_intervals_from_schedule, max_coverage_paths_inclusive, fill_sos_counter_manning, prefix_non_zero
from backend_algo import get_officer_last_counter_and_empty_counters, update_main_officers_schedule_last_counter,find_empty_rows, slot_officers_matrix_gap_aware, merge_prefixed_matrices, plot_officer_timetable_with_labels
# === user inputs ===
main_officers_reported = '1-18'
report_gl_counters = "4AC1, 8AC11, 12AC21, 16AC31"
sos_timings = '1000-1300, 2000-2200, 1315-1430;2030-2200,1315-1430;2030-2200, 1000-1130;1315-1430;2030-2200, 1200-2200, 1400-1830, 1400-1830, 1630-1830,1330-2200,1800-2030, 1800-2030, 1730-2200, 1730-1900, 1700-1945'
ro_ra_officers = "3RO2100, 11RO1700,15RO2130"


main_officers_template = init_main_officers_template()
main_officers_schedule, reported_officers, valid_ro_ra = generate_main_officers_schedule(main_officers_template, main_officers_reported, report_gl_counters, ro_ra_officers )
counter_matrix_wo_last, counter_no = officer_to_counter_matrix(main_officers_schedule)
officer_last_counter, empty_counters_2030 = get_officer_last_counter_and_empty_counters(
reported_officers, valid_ro_ra, counter_matrix_wo_last)
updated_main_officers_schedule = update_main_officers_schedule_last_counter(
    main_officers_schedule, officer_last_counter, empty_counters_2030
)
counter_matrix, counter_no = officer_to_counter_matrix(updated_main_officers_schedule)
counter_w_partial_availability = find_partial_availability(counter_matrix)
officer_names, base_schedules = build_officer_schedules(sos_timings)
all_break_schedules = generate_break_schedules(base_schedules, officer_names)
chosen_schedule_indices, best_work_count, min_penalty = greedy_smooth_schedule_beam(
    base_schedules,None,all_break_schedules,beam_width=20)
sos_schedule_matrix = generate_sos_schedule_matrix(chosen_schedule_indices, all_break_schedules, officer_names)
schedule_intervals_to_officers, schedule_intervals = get_intervals_from_schedule(sos_schedule_matrix)
chains = greedy_longest_partition_inclusive(schedule_intervals)
paths = max_coverage_paths_inclusive(chains)
print("=== best work count ===")
print(best_work_count)
sos_counter_manning = fill_sos_counter_manning(counter_matrix, paths, schedule_intervals_to_officers)
prefixed_counter_matrix = prefix_non_zero(counter_matrix, "M")
empty_rows, partial_empty_rows, partial_empty_rows_index = find_empty_rows(counter_matrix)
prefixed_sos_counter_manning = slot_officers_matrix_gap_aware(schedule_intervals_to_officers, partial_empty_rows)
merged = merge_prefixed_matrices(prefixed_counter_matrix, prefixed_sos_counter_manning)
# Example usage:
fig1 = plot_officer_timetable_with_labels(merged)
fig1.show()  # show in Jupyter or browser

fig2 = plot_officer_timetable_with_labels(counter_matrix)
fig2.show()


[(3, 'RO', '2100'), (11, 'RO', '1700'), (15, 'RO', '2130')]
=== best work count ===
[ 2  2  2  2  2  1  1  1  2  2  2  2  2  4  4  4  7  7  7  4  1  1  4  4
  4  4  4  4  4  4  7  7 10 10  7  4  4  4  6  6  6  6  6  7  7  7  7  7]
[8, 37, 26, 16, 35, 25, 15, 5, 34, 24, 14, 4, 33, 23, 13, 3, 32, 22, 12, 2]
Available empty counters (0-indexed) after filtering: [7, 36, 25, 15, 34, 24, 14, 4, 33, 23, 13, 3, 32, 22, 12, 2, 31, 21, 11, 1]
Available empty counters (display names): ['C8', 'C37', 'C26', 'C16', 'C35', 'C25', 'C15', 'C5', 'C34', 'C24', 'C14', 'C4', 'C33', 'C23', 'C13', 'C3', 'C32', 'C22', 'C12', 'C2']
Partial counters: [0, 5, 6, 9, 10, 17, 20, 26, 30, 35, 37]

Assignment Statistics:
  Partial counters used: 9/11
  New empty counters used: 3
  Total counters used: 12/41

Partial Counter Validation:
  Counter 0: available [(5, 47)], assigned 6 intervals
    (7,12) officer 0 ✓
    (13,18) officer 2 ✓
    (22,27) officer 6 ✓
    (28,33) officer 14 ✓
    (36,39) officer 14 ✓
    (40,4

In [2]:
def assign_ac_counters_to_main_schedule(main_officers_schedule, report_gl_counters):
    """
    Assign AC counters to main officers' schedule based on report_gl_counters.

    Args:
        main_officers_schedule: dict {officer_key: list of 48 slots}
        report_gl_counters: str, e.g., "4AC5, 6AC7"

    Returns:
        updated_main_officers_schedule: dict with updated counter assignments
    """
    updated_schedule = {k: v.copy() for k, v in main_officers_schedule.items()}

    # Split input string and clean whitespace
    report_list = [s.strip() for s in report_gl_counters.split(',')]

    for officer_counter in report_list:
        if "AC" in officer_counter:
            idx = officer_counter.index("AC")
            officer_id = int(officer_counter[:idx])
            counter_no = int(officer_counter[idx + 2:])

            # Only apply to officers divisible by 4
            if officer_id % 4 == 0:
                officer_key = "M" + str(officer_id)
                if officer_key in updated_schedule:
                    # Assign counter_no to slots 0 to 4 inclusive
                    for slot in range(0, 5):
                        updated_schedule[officer_key][slot] = counter_no

    return updated_schedule




In [3]:

report_gl_counters = "4AC5, 6AC7"

updated_schedule = assign_ac_counters_to_main_schedule(main_officers_schedule, report_gl_counters)
print(main_officers_schedule["M4"])
print(updated_schedule["M4"][:10])


[ 1  1  1  1  1  0 40 40 40 40 40 40  0  0 30 30 30 30 30 30 30 30 30 30
  0  0  0 20 20 20 20 20 20 20 20 20  0  0  0 41 41 41 41 41 41 41 41 41]
[ 5  5  5  5  5  0 40 40 40 40]
